In [ ]:
struct_def = """"""

In [ ]:
def is_field_def(line):
    if not line or line.startswith('struct') or line.startswith('//') or line.startswith('{') or line.startswith('}'):
        return False
    return True

def detect_field_size(line):
    if '[]' in line:
        return 0x20
    
    size_0x20_types_list = ['uint256', 'bytes32', 'int256', 'bytes']
    if any(map(lambda x: line.startswith(x), size_0x20_types_list)):
        return 0x20
    
    size_0x01_types_list = ['byte', 'bool', 'uint8', 'int8']
    if any(map(lambda x: line.startswith(x), size_0x01_types_list)):
        return 1

def is_size_comment(line):
    if line.startswith('// 0x'):
        return True
    return False
    
def get_field_name(line):
    for subs in line.split():
        if subs.endswith(';'):
            return subs.replace(';', '').upper()

result_struct_def = ''
current_offset = 0
offset_map = dict()
for raw_line in struct_def.splitlines():
    line = raw_line.strip()
    if is_field_def(line):
        offset_map[get_field_name(line) + '_OFFSET'] = hex(current_offset)
        result_struct_def += f'// {hex(current_offset)}\n'
        current_offset += detect_field_size(line)
    if not is_size_comment(line):
        result_struct_def += line + '\n'

print(result_struct_def)
for k, v in offset_map.items():
    print(f'uint256 constant {k} =', f'{v};')